In [1]:
from db_info import user, password, host, port, database
import pandas as pd
import mysql.connector

db_connection = mysql.connector.connect(
user = user,
password = password,
host = host,
port = port,
database = database,
)

cursor = db_connection.cursor()

cursor.execute("select * from injury_content")

result = cursor.fetchall()

df = pd.DataFrame(result, columns = [i[0] for i in cursor.description])
df.head()

,판례정보일련번호,사건번호,선고,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용
0,241209,2022두51031,선고,판결,[1] 국내사업장을 가진 외국법인이 국내 및 국외에 걸쳐 선박에 의한 국제운송업을...,[1] 구 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것...,[1] 구 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것...,"[3] 대법원 1994. 8. 12. 선고 92누12094 판결(공1994하, 2...","【원고, 상고인 겸 피상고인】 ○○○서비스 (소송대리인 법무법인 케이원챔버 담당변호..."
1,240925,2023도18812,선고,판결,특수상해죄 및 특수협박죄의 구성요건 중 위험한 물건을 ‘휴대하여’의 의미 / 이...,"형법 제258조의2 제1항, 제257조 제1항, 제284조, 제283조 제1항은 위...","형법 제257조 제1항, 제258조의2 제1항, 제283조 제1항, 제284조<br/>","대법원 2002. 6. 14. 선고 2002도1341 판결, 대법원 2004. 6....",【피 고 인】 피고인<br/>【상 고 인】 피고인 및 검사<br/>【원심판결】 대전...
2,240809,2021도6357,선고,전원합의체 판결,형사소송법 제33조 제1항 제1호에서 필요적 국선변호인 선정사유 중 하나로 정한 ‘...,[다수의견] 형사소송법 제33조 제1항 제1호는 피고인에게 변호인이 없는 때에 ...,"헌법 제12조, 형사소송법 제33조 제1항 제1호, 제69조, 제473조 제1항, ...","대법원 2009. 5. 28. 선고 2009도579 판결(공2009하, 1060)(...",【피 고 인】 피고인<br/>【상 고 인】 피고인<br/>【변 호 인】 변호사 유소...
3,239933,2023도11371,선고,판결,[1] 법원이 피해자 등을 공판기일에 출석하게 하여 형사소송법 제294조의2 제...,[1] 법원은 피해자 등의 신청이 있는 때에 그 피해자 등을 증인으로 신문하여야...,"[1] 형사소송법 제294조의2 제1항, 제2항, 형사소송규칙 제134조의10 제1...",None,【피 고 인】 피고인<br/>【상 고 인】 피고인<br/>【변 호 인】 변호사 신영...
4,241179,2023도12851,선고,판결,형의 집행 및 수용자의 처우에 관한 법률에 따른 징벌을 받아 그 집행을 종료한 ...,None,"헌법 제13조 제1항, 형의 집행 및 수용자의 처우에 관한 법률 제107조, 제10...","대법원 2000. 10. 27. 선고 2000도3874 판결(공2000하, 2487...",【피 고 인】 피고인<br/>【상 고 인】 피고인<br/>【변 호 인】 변호사 홍푸...


# 참조조문 전처리

In [2]:
df['참조조문']=df['참조조문'].fillna('참조조문 없음')


In [12]:
df['참조조문'].value_counts()

참조조문
참조조문 없음                                                                               803
민법 제750조                                                                               84
민법 제763조                                                                               68
자동차손해배상보장법 제3조                                                                         37
형사소송법 제308조                                                                            27
                                                                                     ... 
가.나.다.라.근로기준법 제27조 제1항 / 다. 구 근로기준법 제27조의2                                              1
가.        민법 제758조 제1항, 나.        제763조,         제393조,         상법 제727조                1
민법 제750조,         제763조,         제396조,         교통사고처리특례법 제3조,         민사소송법 제183조      1
형사소송법 제307조, 제308조                                                                      1
지방자치법 제102조,         국가배상법 제2조                                                          1
Name:

In [ ]:
import re
def extract_laws(text):
    if isinstance(text, str):  # 텍스트가 문자열인지 확인
    # '법', '시행령', '에 관한 법률', '규칙'으로 끝나는 단어를 정규 표현식으로 추출
        laws = re.findall(r'\b(\w+(?:법|시행령|에 관한 법률|규칙))\b', text)
        return set(laws)
    return set()

# 각 참조조문에서 법과 시행령 이름을 찾기
law_names = set()
for reference in df['참조조문']:
    law_names.update(extract_laws(reference))


# 법과 시행령 이름을 붙이는 함수
def add_law_prefix(x):
    if isinstance(x, str):  
        previous_law = None  # 이전 법 이름 저장
        new_references = []

        for part in x.split(','):
            part = part.strip()
            part = re.sub(r'[\[\]［］].*?[\]\］]', '', part)
            part = re.sub(r'\s*[가-힣]\.\s*', '', part)
            part = part.strip('<br/>')
            part = part.replace('/', ',')
            part = re.sub(r'\s+', ' ', part).strip()

            # 현재 조각에 법 이름이 있는 경우, previous_law 업데이트
            for law_name in law_names:
                if law_name in part:
                    previous_law = law_name  # 이전 법 이름을 현재 법 이름으로 설정
                    break

            # 법 이름이 없고, 조를 포함하는 경우에만 이전 법 이름 붙이기
            if previous_law and not any(law_name in part for law_name in law_names):
                if re.search(r'\d+조', part):  # '조'를 포함하는 경우
                    part = f"{previous_law} {part}"

            new_references.append(part)

        # 결과 조각들을 다시 쉼표로 결합하여 반환
        return ', '.join(new_references)
    return x  # 문자열이 아닐 경우 그대로 반환


In [23]:
law_names

{'가사소송법',
 '같은법',
 '같은법시행규칙',
 '같은법시행령',
 '거래신고에 관한 법률',
 '건널목개량촉진법',
 '건설기술관리법',
 '건설기술관리법시행령',
 '건설산업기본법',
 '건설산업기본법시행령',
 '건설업법',
 '건설업법시행령',
 '건축법',
 '건축법시행령',
 '검사규칙',
 '검찰집행사무규칙',
 '검찰청법',
 '게임산업진흥에 관한 법률',
 '경범죄처벌법',
 '경비업법',
 '경찰공무원법',
 '경찰관직무집행법',
 '경찰범처벌규칙',
 '고압가스안전관리법',
 '공동주택관리법',
 '공무원및사립학교교직원의료보험법',
 '공무원연금법',
 '공정거래에 관한 법률',
 '공직선거및선거부정방지법',
 '공직선거법',
 '관광진흥법',
 '관리에 관한 법률',
 '관세법',
 '광업법',
 '교육공무원법',
 '교육기본법',
 '교육법',
 '교통사고처리특례법',
 '구국가배상법',
 '구조개선에 관한 법률',
 '국가공무원법',
 '국가배상법',
 '국가배상법시행령',
 '국가보상법',
 '국가보안법',
 '국가소송법',
 '국가유공자등예우및지원에관한법률시행령',
 '국가재정법',
 '국민건강보험법',
 '국민의료보험법',
 '국민체육진흥법',
 '국바배상법',
 '국유철도건설규칙',
 '국유철도운전규칙',
 '국적법',
 '국제사법',
 '국회의원선거법',
 '군법회의법',
 '군사법원법',
 '군사원호보상급여금법',
 '군사원호보상법',
 '군수품관리법',
 '군수품관리법시행령',
 '군인사법',
 '군인연금법',
 '군형법',
 '귀속재산처리법',
 '규제에 관한 법률',
 '근로기준법',
 '근로기준법시행령',
 '근로복지기본법',
 '기본법',
 '난민법',
 '노동관계조정법',
 '노동위원회법',
 '노동쟁의조정법',
 '노동조합및노동관계조정법',
 '노동조합법',
 '노인복지법',
 '농어촌도로정비법',
 '농업협동조합법',
 '농지개혁법',
 '단속법',
 '대책에 관한 법률',
 '도로교통법',
 '도로교통

In [14]:
reason=df['참조조문'].apply(add_law_prefix)


0       구 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것) 제91...
1       형법 제257조 제1항, 형법 제258조의2 제1항, 형법 제283조 제1항, 형법...
2       헌법 제12조, 형사소송법 제33조 제1항 제1호, 형사소송법 제69조, 형사소송법...
3                            성폭력범죄의 처벌 등에 관한 특례법 제14조 제1항
4                                                 참조조문 없음
                              ...                        
4184                                              참조조문 없음
4185                                              참조조문 없음
4186                                              참조조문 없음
4187               법원재난에 기인한 민형사사건 임시조치법 제1조, 군법회의법 제432조
4188                               지방자치법 제102조, 국가배상법 제2조
Name: 참조조문, Length: 4189, dtype: object

In [106]:
# 데이터프레임의 '참조조문' 열에 전처리 적용
df['참조조문'] = df['참조조문'].apply(add_law_prefix)

In [134]:
df['참조조문'][df['참조조문'].str.contains(', 제2항,')]

2       헌법 제12조, 형사소송법 제33조 제1항 제1호, 형사소송법 제69조, 형사소송법...
6       형사소송법 제33조 제1항, 형사소송법 제361조의2 제1항, 제2항, 형사소송법 ...
101     민법 제396조, 민법 제750조, 민법 제751조, 민법 제763조, 병역법 제1...
165     도로교통법 제41조 제1항, 제2항, 도로교통법 제107조의2 제1호, 제2호, 도...
189     형사소송법 제294조의2 제1항, 제2항, 형사소송규칙 제134조의10 제1항, 형...
235     형법 제21조 제1항, 제2항, 형법 제22조 제1항, 제3항, 형법 제260조 제...
325     형법 제136조 제1항, 형사소송법 제211조, 형사소송법 제212조, 경찰관 직무...
384     헌법 제12조 제4항, 헌법 제27조, 형사소송법 제266조의3 제1항 제4호, 제...
394     교통사고처리특례법 제3조 제1항, 제2항, 형법 제268조, 도로교통법 제25조 제...
454     의료법 제53조 제1항, 제2항, 의료법 제59조 제1항, 행정소송법 제27조, 의...
568     교통사고처리 특례법 제4조 제1항, 교통사고처리 특례법 제1조, 특례법 제4조 제1...
598     국제사법 제2조 제1항, 제2항, 국제사법 제2조 제1항, 제2항, 민사소송법 제2...
604     행정소송법 제1조, 행정소송법 제2조, 행정소송법 제4조, 행정소송법 제19조, 행...
650         경찰관직무집행법 제1조, 경찰관직무집행법 제3조 제1항, 제2항, 제3항, 제7항
665     교통사고처리 특례법 제1조, 특례법 제4조 제1항, 제2항, 교통사고처리 특례법 제...
667     해상교통안전법시행규칙 제8조, 상법 제674조, 상법 제682조, 민법 제393조 ...
679     헌법 제13조 제1항, 형법 제1조 제1항, 성폭력범죄의 처벌 등에 관한 특례법 제...
713     헌법 제21

In [114]:
#조문 참조 정도 확인
refer_clause=[]
for clause in df['참조조문']:
    parts = clause.split(',')
    parts = [part.strip() for part in parts if part.strip()]
    for sub in parts:
        refer_clause.append(sub)
refer_clause = [item for item in refer_clause if item.strip()]
refcla=pd.DataFrame(data=refer_clause)
refcla.value_counts()

0               
참조조문 없음             803
민법 제763조            422
민법 제750조            391
민법 제393조            233
형사소송법 제308조         181
                   ... 
민사소송법 제193조 1항 4      1
민사소송법 제193조 2항        1
민사소송법 제198조           1
민사소송법 제201조           1
회사정리법 제96조            1
Name: count, Length: 3350, dtype: int64

In [113]:
df['참조조문'].value_counts()

참조조문
참조조문 없음                                               803
민법 제750조                                               86
민법 제763조                                               71
자동차손해배상보장법 제3조                                         37
민법 제763조, 민법 제393조                                     30
                                                     ... 
형법 제37조, 형법 제337조, 형사소송법 제298조                          1
형법 제335조, 형법 제333조                                      1
특정범죄가중처벌등에관한법률 제5조의5, 특정강력범죄의처벌에관한특례법 제3조, 형법 제35조      1
근로기준법 제33조, 근로기준법 제40조                                  1
지방자치법 제102조, 국가배상법 제2조                                  1
Name: count, Length: 2473, dtype: int64

In [123]:
# 리스트를 데이터프레임으로 변환
refcla = pd.DataFrame(data=refer_clause, columns=['조문'])

# 빈도 계산 결과를 데이터프레임으로 변환하여 모든 값 출력
refcla_counts_df = refcla.value_counts().reset_index(name='빈도')
refcla_counts_df.columns = ['조문', '빈도']


pd.set_option('display.max_rows',60)


refcla_counts_df  

,조문,빈도
0,참조조문 없음,803
1,민법 제763조,422
2,민법 제750조,391
3,민법 제393조,233
4,형사소송법 제308조,181
...,...,...
3345,민사소송법 제193조 1항 4,1
3346,민사소송법 제193조 2항,1
3347,민사소송법 제198조,1
3348,민사소송법 제201조,1


In [141]:
# 이전 법 이름, 조, 항 정보를 붙이는 함수
# 항과 호에 법, 조, 항을 붙이는 함수
def add_clause_prefix(parts):
    previous_law = None
    previous_clause = None
    previous_article = None
    updated_parts = []

    for part in parts:
        # 항으로 끝나는 경우
        if re.search(r'\d+항$', part):
            # 앞에 '법'과 '조'가 없는 경우
            if previous_clause and previous_law and not any(name in part for name in [previous_law, previous_clause] if name):
                part = f"{previous_law} {previous_clause} {part}"
            previous_article = part  # 항 정보 저장

        # 호로 끝나는 경우
        elif re.search(r'\d+호$', part):
            # 앞에 '법', '조', '항'이 모두 없는 경우
            if previous_clause and previous_article and previous_law and not any(name in part for name in [previous_law, previous_clause, previous_article] if name):
                part = f"{previous_law} {previous_clause} {previous_article} {part}"
            # 앞에 '항'만 있는 경우
            elif previous_clause and previous_law and previous_article and (previous_article in part) and not any(name in part for name in [previous_law, previous_clause] if name):
                part = f"{previous_law} {previous_clause} {part}"

        # 조로 끝나는 경우, 이전 법 이름과 조 정보 업데이트
        elif re.search(r'\d+조', part):
            previous_clause = part  # 조 정보 저장
            previous_article = None  # 항 정보 초기화

        # 법 이름이 있는 경우, previous_law 업데이트
        for law_name in law_names:
            if law_name in part:
                previous_law = law_name  
                break

        updated_parts.append(part)

    return updated_parts


# 2단계: 각 참조조문을 분리하고 항과 호에 앞의 정보 추가
refer_clause = []
for clause in df['참조조문']:
    parts = clause.split(',')
    parts = [part.strip() for part in parts if part.strip()]
    refer_clause.extend(add_clause_prefix(parts))

# 3단계: 분리된 조문을 데이터프레임에 저장하고 빈도 계산
refer_clause = [item for item in refer_clause if item.strip()]
refcla = pd.DataFrame(data=refer_clause, columns=['조문'])
refcla_counts = refcla.value_counts()
print(refcla_counts)

조문                                                                                                                                                                                                  
참조조문 없음                                                                                                                                                                                                 803
민법 제763조                                                                                                                                                                                                422
민법 제750조                                                                                                                                                                                                391
민법 제393조                                                                                                                                                                                       

내일시도

In [5]:
import re

# 법과 시행령 이름을 추출하는 함수
def extract_laws(text):
    if isinstance(text, str):  # 텍스트가 문자열인지 확인
        # '법' 또는 '시행령'으로 끝나는 단어를 정규 표현식으로 추출
        laws = re.findall(r'\b(\w+(?:법|시행령|에 관한 법률|규칙))(?=\s|$)', text)
        return set(laws)
    return set()

# 법과 시행령 이름을 수집
law_names = set()
for reference in df['참조조문']:
    law_names.update(extract_laws(reference))

# 법 이름과 조, 항을 붙이는 함수
def add_law_prefix(x):
    if isinstance(x, str):  
        previous_law = None   # 이전 법 이름 저장
        previous_clause = None  # 이전 조 저장
        previous_article = None  # 이전 항 저장
        new_references = []

        for part in x.split(','):
            part = part.strip()
            part = re.sub(r'[\[\]［］].*?[\]\］]', '', part)
            part = re.sub(r'\s*[가-힣]\.\s*', '', part)
            part = part.strip('<br/>')
            part = part.replace('/', ',')
            part = re.sub(r'\s+', ' ', part).strip()

            # 현재 조각에 법 이름이 있는 경우, previous_law 업데이트
            for law_name in law_names:
                if law_name in part:
                    previous_law = law_name
                    break

            # 조로 끝나는 경우 법 이름과 조항을 저장
            if re.search(r'\d+조$', part):
                if previous_law and previous_law not in part:
                    part = f"{previous_law} {part}"  # 법 이름을 붙임
                previous_clause = part
                previous_article = None  # 항 정보 초기화

            # 항으로 끝나는 경우, 앞에 법과 조가 없는 경우 추가
            elif re.search(r'\d+항$', part):
                if previous_clause and previous_law and not any(name in part for name in [previous_law, previous_clause]):
                    part = f"{previous_law} {previous_clause} {part}"
                previous_article = part  # 항 정보 저장

            # 호로 끝나는 경우
            elif re.search(r'\d+호$', part):
                # 법, 조, 항이 모두 없는 경우 추가
                if previous_clause and previous_article and previous_law and not any(name in part for name in [previous_law, previous_clause, previous_article]):
                    part = f"{previous_law} {previous_clause} {previous_article} {part}"
                # 항만 있고 법과 조가 없는 경우
                elif previous_clause and previous_law and previous_article and (previous_article in part) and not any(name in part for name in [previous_law, previous_clause]):
                    part = f"{previous_law} {previous_clause} {part}"

            new_references.append(part)

        # 결과 조각들을 다시 쉼표로 결합하여 반환
        return ', '.join(new_references)
    return x

# df에 함수 적용
clause= df['참조조문'].apply(add_law_prefix)
clause.value_counts()

참조조문
참조조문 없음                                               803
민법 제750조                                               86
민법 제763조                                               71
자동차손해배상보장법 제3조                                         37
민법 제763조, 민법 제393조                                     30
                                                     ... 
사회보호법 제5조 제1항 , 동법부칙 제2조, 형법 제12조                       1
형법 제37조, 제337조 ,형사소송법 제298조                             1
형법 제335조, 형법 제333조                                      1
특정범죄가중처벌등에관한법률 제5조의5, 특정강력범죄의처벌에관한특례법 제3조, 형법 제35조      1
지방자치법 제102조, 국가배상법 제2조                                  1
Name: count, Length: 2497, dtype: int64

In [105]:
pd.set_option('display.max_rows',None)


refcla_counts  

NameError: name 'refcla_counts' is not defined

# 여기부터 참조조문 전처리 시작

# 참조조문 깔끔하게 구조 통일

1.',' 단위로 나눠지는 것과 [1]~~~ /[2]~~~ 이런식으로 나뉜게 있음.

2. [1]은 지우고 /[2] 이런식으로 되는건 ,로 대체하기.

3. 가.나.다.라. 와 같이 자음+'ㅏ'+'.'의 조합이 있으므로 이를 제외 '(가|나|다|라|마|바|사|아|자|차|카|타|파|하)\\.'

4. []와 같은 일반대괄호 말고 전각대괄호로 되어있는 것 나와서 이거 제거

In [5]:
import re


import re


def clean_clause(text): # 전부 , 단위로 분리될수 있게 변환

    text=text.replace('<br/>',',')

    # 각 그룹을 '/'로 나누어 처리
    groups = text.split('/')
    results = []

    for group in groups:
        group = group.strip()  # 각 그룹의 양쪽 공백 제거
        group=re.sub(r'(가|나|다|라|마|바|사|아|자|차|카|타|파|하)\.', '', group) # 가., 나., 다., 라., 마., 바., 사., 아., 자., 차., 카., 타., 파., 하. 댜. 형태만 제거
        group=re.sub(r'(갸|냐|댜|랴|먀|뱌|샤|야|쟈|챠|캬|탸|퍄|햐)\.', '', group) # 가나다라의 오타도 제거
        
        group=re.sub(r'[\[\［]\d+[\]\］]\s*', '', group)  # '[1]', '[2]' 형식 제거
        group = group.strip()  # 각 그룹의 양쪽 공백 제거
        
        # 쉼표로 나눠 조각을 분리
        parts = re.split(r',\s*', group)                

        transformed_parts = []

        for part in parts:    # 조, 항, 호 뒤에 문자가 붙어 있는 경우 처리
            
            if re.search(r'([0-9]+조|[0-9]+항|[0-9]+호)([^\s])', part):
                part = re.sub(r'([0-9]+조|[0-9]+항|[0-9]+호)([^로의\s])', r'\1, \2', part)            
            part = part.strip()
            # 변환된 조문 추가
            transformed_parts.append(part)

        # 각 그룹을 쉼표로 연결하고 결과에 추가
        if transformed_parts:  # transformed_parts가 비어있지 않으면
            results.append(', '.join(transformed_parts))

    # 전체 결과를 반환
    return ', '.join(results)

# 예시 텍스트들에 대해 적용
examples = [
' [1] 구 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것) 제91조 제1항 제3호, 제93조 제5호, 제11호 (차)목[현행 제93조 제10호 (타)목 참조], 법인세법 시행령 제132조 제2항 제7호 / [2] 행정소송법 제26조[증명책임] / [3] 구 부가가치세법(2010. 1. 1. 법률 제9915호로 개정되기 전의 것) 제10조 제2항 제2호(현행 제20조 제1항 제2호 참조), 제11조 제1항 제3호(현행 제23조 제1항 참조), 제2항(현행 제25조 제1항 참조), 구 부가가치세법 시행령(2013. 6. 28. 대통령령 제24638호로 전부 개정되기 전의 것) 제25조 제1항(현행 제32조 제1항 참조), 행정소송법 제26조[증명책임]',
'가.나.라.        산업재해보상보험법 제3조제1항 / 다.         민법 제756조민사소송법'
]

for example in examples:
    print(clean_clause(example))


구 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것) 제91조 제1항 제3호, 제93조 제5호, 제11호 (차)목[현행 제93조 제10호 (타)목 참조], 법인세법 시행령 제132조 제2항 제7호, 행정소송법 제26조, [증명책임], 구 부가가치세법(2010. 1. 1. 법률 제9915호로 개정되기 전의 것) 제10조 제2항 제2호, (현행 제20조 제1항 제2호 참조), 제11조 제1항 제3호, (현행 제23조 제1항 참조), 제2항, (현행 제25조 제1항 참조), 구 부가가치세법 시행령(2013. 6. 28. 대통령령 제24638호로 전부 개정되기 전의 것) 제25조 제1항, (현행 제32조 제1항 참조), 행정소송법 제26조, [증명책임]
산업재해보상보험법 제3조, 제1항, 민법 제756조, 민사소송법


In [7]:
ccl=df['참조조문'].apply(clean_clause)
ccl = ccl.apply(lambda x: x.replace(', ,', ',') if isinstance(x, str) else x)
ccl

0       구 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것) 제91...
1           형법 제257조 제1항, 제258조의2 제1항, 제283조 제1항, 제284조, 
2       헌법 제12조, 형사소송법 제33조 제1항 제1호, 제69조, 제473조 제1항, ...
3       형사소송법 제294조의2 제1항, 제2항, 형사소송규칙 제134조의10 제1항, 제...
4       헌법 제13조 제1항, 형의 집행 및 수용자의 처우에 관한 법률 제107조, 제10...
                              ...                        
4184                                           형법 제161조, 
4185                                        형사소송법 제311조, 
4186                        형사소송법 제307조, 제308조, 형법 제10조, 
4187                                              참조조문 없음
4188    조치령 제1조, 제4조, 제4조 3호, 형사소송법 제516조, 제520조, 형사소송...
Name: 참조조문, Length: 4189, dtype: object

In [45]:
df.iloc[4188]

판례정보일련번호                                                85842
사건번호                                                  4284형상1
선고                                                         선고
판결유형                                                       판결
판시사항        가. \n          비상사태하 범죄처벌에 관한 특별조치령 제4조 3호의 「관...
판결요지        가. \n          조치령 제4조 제3호전단의 관헌참칭은 체포감금등과 대등되...
참조조문        조치령 제1조, \n        제4조, \n        제4조 3호, \n\n...
참조판례                                                     None
판례내용        【비상상고인】   검찰총장<br/>【원    심】 \n대구지방법원<br/>【주   ...
Name: 4188, dtype: object

# 참조조문 법이름 전처리

In [239]:
def extract_law_names(series):
    law_names_set=set()
    pattern = r'([가-힣]+?(법\s*시행령|시행령|에\s*관한\s*법률|시행규칙|규칙|협정|의사록|관례|고시|대통령령|경제명령|법)(\([^)]*\))?)'
    # 정규 표현식을 사용하여 괄호를 기준으로 분리
    
    def split_outside_parentheses(text):
        result = []
        start = 0
        stack = []  # 괄호 스택

        for i, char in enumerate(text):
            if char in "{(":
                stack.append(char)  # 괄호 열기
            elif char in "})" and stack:
                stack.pop()  # 괄호 닫기
            elif char == "," and not stack:
                # 스택이 비어 있을 때의 쉼표는 분리 기준
                result.append(text[start:i].strip())
                start = i + 1

        # 마지막 남은 부분 추가
        result.append(text[start:].strip())
        return result

    for text in series:
        # 1. 쉼표로 문자열을 분리 (괄호 안 쉼표 무시)
        parts = split_outside_parentheses(text)
        for part in parts:
            # 불필요한 \n 및 공백 제거
            part = re.sub(r'\n+', ' ', part)
            part = re.sub(r'\s+', ' ', part)  # 여러 공백을 하나로 줄임
            part = re.sub(r'[\[\]]', '', part)  # [] 괄호 제거
            part = re.sub(r'〔\d+〕', '', part)
            part = part.strip()
            part = re.sub(r'\{[^}]*\}', '', part)  # 중괄호 안의 내용 제거

            split_pattern = r'([^()]*)(\s*\([^()]*\))?'

            matches = re.findall(split_pattern, part)
            matches = [(off_maren.replace(',', '').strip(), maren.strip()) for off_maren, maren in matches]

            for match in matches:
                before_parentheses = match[0].strip()  # 괄호 이전 내용
                in_parentheses = match[1].strip() if match[1] else ''  # 괄호 내용
                # '숫자. ' 형식이 있다면 제거
                before_parentheses = re.sub(r'^\d+\.\s*', '', before_parentheses)
                # print('in: ',in_parentheses)

                # 법 이름 추출: 중괄호가 제거된 `before_parentheses`에서 패턴 찾기
                match = re.search(pattern, before_parentheses)
                if match:
                    # 패턴이 일치하는 위치까지 추출
                    law_name = before_parentheses[:match.end()] + in_parentheses
                    if len(law_name) > 1:  # 길이가 1보다 큰 경우만 추가
                        law_names_set.add(law_name)

    return list(law_names_set)


# 결과 출력
law_names = extract_law_names(ccl)
print('추출된 법 이름:', law_names)
extract_law_names(pd.Series(ccl[0]))

추출된 법 이름: ['구 장애인차별금지 및 권리구제 등에 관한 법률(2012. 10. 22. 법률 제11522호로 개정되기 전의 것)', '보험사기방지 특별법', '선박법', '신탁법', '구 출입국관리법 시행령(2013. 6. 21. 대통령령 제24628호로 개정되기 전의 것)', '구 영유아보육법(2017. 3. 14. 법률 제14597호로 개정되기 전의 것)', '형사보상 및 명예회복에 관한 법률', '마약류관리에 관한 법률(나)', '형사보상법', '아동·청소년의 성보호에 관한 법률(2010. 4. 15. 법률 제10260호, )', '구 학교안전사고 예방 및 보상에 관한 법률(2011. 12. 31. 법률 제11141호로 개정되기 전의 것)', '구 형법(2013. 4. 5. 법률 제11731호로 개정되기 전의 것)', '사행행위 등 규제 및 처벌 특례법', '신용장통일규칙(1993년 제5차 개정된 것)', '구 국가유공자등예우및지원에관한법률(1998. 5. 9. 대통령령 제15792호로 개정되기 전의 것)', '총포·도검·화약류단속법', '구 도로교통법(2005. 5. 31. 법률 제7545호로 전부 개정되기 전의 것)', '구 국회에서의 증언·감정 등에 관한 법률(2010. 3. 12. 법률 제10051호로 개정되기 전의 것)', '자동차손해배상 보장법', '구 초·중등교육법(2012. 3. 21. 법률 제11384호로 개정되기 전의 것)', '구 형법(2012. 12. 18. 법률 제11574호로 개정되기 전의 것)', '승강기시설 안전관리법', '교육기본법', '식품위생법', '구 자동차손해배상 보장법 시행령(2021. 1. 5. 대통령령 제31380호로 개정되기 전의 것)', '장애인복지법', '구 도로교통법(2014. 12. 30. 법률 제12917호로 개정되기 전의 것)', '국제항공운송에 있어서의 일부규칙(개정 바르샤바 협약)', '군수품관리법시행령', '구 직업안정및고용촉진에관한법률(1994. 1. 7. 법률 제4733호 직업안정법으로

['구 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것)',
 '구 부가가치세법 시행령(2013. 6. 28. 대통령령 제24638호로 전부 개정되기 전의 것)',
 '구 부가가치세법(2010. 1. 1. 법률 제9915호로 개정되기 전의 것)',
 '행정소송법',
 '법인세법 시행령']

241107 법이름을 시행령까지 추출

ㄴ> 법 이름에서 '구 ' 랑 법 이름 뒤에 붙는 괄호들을 지우기/
ㄴ> '구 법' 뒤에 조 항 호 뒤에 괄호 안에 현행 참조가 나오면 그걸 꺼내와서 따로 현행 칼럼 만들기

In [224]:
extract_law_names(ccl[0])

[]

In [219]:
law_names

['상표법(2014. 6. 11. 법률 제12751호로 개정되기 전의 것)',
 '선박법',
 '신탁법',
 '성보호에 관한 법률(2010. 4. 15. 법률 제10260호, )',
 '합의의사록',
 '참여에 관한 법률',
 '건설업법시행령(1997. 7. 10. 대통령령 제15433호 건설산업기본법시행령으로 개정되기 전의 것)',
 '마약류관리에 관한 법률(나)',
 '화약류등단속법시행령',
 '형사보상법',
 '운영규칙(2010. 6. 30. 경찰청훈령 제595호로 개정되기 전의 것)',
 '관리에 관한 법률(나)',
 '신용장통일규칙(1993년 제5차 개정된 것)',
 '등에 관한 법률(2022. 12. 27. 법률 제19104호로 개정되기 전의 것)',
 '참여에 관한 법률(2012. 1. 17. 법률 제11155호로 개정되기 전의 것)',
 '영유아보육법(2017. 3. 14. 법률 제14597호로 개정되기 전의 것)',
 '교육기본법',
 '식품위생법',
 '장애인복지법',
 '산업안전보건법(2002. 12. 30. 법률 제6847호로 개정되기 전의 것)',
 '처벌에 관한 법률(2014. 12. 30. 법률 제12896호로 개정되기 전의 것)',
 '부착에 관한 법률',
 '군수품관리법시행령',
 '섭외사법',
 '군형법(2013. 4. 5. 법률 제11734호로 개정되기 전의 것)',
 '군인연금법',
 '보장법 시행령(1999. 6. 30. 대통령령 제16463호로 개정되기 전의 것)',
 '건설기술관리법',
 '동물보호법(2018. 3. 20. 법률 제15502호로 개정되기 전의 것)',
 '도로교통법(2023. 1. 3. 법률 제19158호로 개정되기 전의 것)',
 '관리에 관한 법률(2012. 6. 7. 대통령령 제23845호로 개정되기 전의 것)',
 '공정거래에 관한 법률(2004. 12. 31. 법률 제7315호로 개정되기 전의 것)',
 '민사집행법',
 '형법(2005. 3. 31. 법률 제7427호로 개정되기 전의 것)',


In [220]:
law_names = [term.replace('에 의한 시설과 구역 및 대한민국에서의 합중국군대의 지위에 관한 협정', '대한민국과 아메리카합중국간의 상호방위조약 제4조에 의한 시설과 구역 및 대한민국에서의 합중국군대의 지위에 관한 협정') for term in law_names]

In [170]:
def clean_extracted_law_names(law_names):
    cleaned_law_names = []
    # 패턴 뒤의 괄호와 '구 ' 접두사 제거
    for law in law_names:
        # '구 ' 접두사 제거
        law = law.replace('구 ', '')
        
        # 패턴 뒤의 괄호 제거
        # 예: "폭력행위 등 처벌에 관한 법률(2014. 12. 30. 법률 제12896호로 개정되기 전의 것)" -> "폭력행위 등 처벌에 관한 법률"
        law = re.sub(r'(법|시행령|에\s*관한\s*법률|규칙|시행규칙|협정|의사록|관례|고시|대통령령|경제명령)(\([^)]*\))', r'\1', law)
        
        cleaned_law_names.append(law.strip())  # 최종적으로 공백 제거 후 리스트에 추가
    
    return cleaned_law_names


In [171]:
law_names_cleaned=clean_extracted_law_names(law_names)

# 참조조문 전처리 완

In [173]:
#최종 법이름
law_names=list(set(law_names))

In [174]:
law_names_cleaned=list(set(law_names_cleaned))

In [187]:
law_names_cleaned

['보험사기방지 특별법',
 '선박법',
 '신탁법',
 '형사보상 및 명예회복에 관한 법률',
 '형사보상법',
 '상가건물 임대차보호법',
 '사행행위 등 규제 및 처벌 특례법',
 '자동차운송사업법',
 '총포·도검·화약류단속법',
 '즉결심판에관한절차법',
 '자동차손해배상 보장법',
 '승강기시설 안전관리법',
 '교육기본법',
 '식품위생법',
 '장애인복지법',
 '직업안정및고용촉진에관한법률',
 '섭외사법',
 '의사상자 예우에 관한 법률',
 '국가유공자 등 예우 및 지원에 관한 법률',
 '성폭력범죄의 처벌 및 피해자보호 등에 관한 법률',
 '군인연금법',
 '건설기술관리법',
 '민사집행법',
 '장애인고용촉진등에관한법률',
 '재해구호법',
 '의료보험법',
 '집회및시위에관한법률',
 '채무자 회생 및 파산에 관한 법률',
 '육운진흥법',
 '약관의규제에관한법률',
 '액화석유가스의 안전 및 사업관리법',
 '혈액관리법',
 '경찰항공 운영규칙',
 '외국인토지법',
 '국민체육진흥법',
 '특정경제범죄 가중처벌 등에 관한 법률',
 '장애인차별금지 및 권리구제 등에 관한 법률',
 '어린이제품 안전 특별법',
 '국바배상법',
 '법무사법',
 '해난심판법',
 '국가를당사자로하는소송에관한법률',
 '관광진흥법',
 '고용보험 및 산업재해보상보험의 보험료 징수 등에 관한 법률',
 '행형법',
 '해사안전법',
 '성폭력범죄자의 성충동 약물치료에 관한 법률',
 '특정 범죄자에 대한 보호관찰 및 전자장치 부착 등에 관한 법률',
 '가 민법',
 '도로교통법',
 '특허법',
 '액화석유가스의 안전관리 및 사업법',
 '특정범죄가중처벌등에 관한 법률',
 '군인사법',
 '국가재정법',
 '국민의료보험법',
 '국민건강보험법',
 '근로자퇴직급여 보장법',
 '총포·도검·화약류등단속법',
 '광업법',
 '폭력행위등처벌에관한법률',
 '금융실명거래및비밀보장에관한법률',
 '도로법',
 '법인세법',
 '국가보안법',
 '노동위원회법

In [181]:
# 법과 시행령 이름을 붙이는 함수 / law_names적용
def add_law_prefix0(x):
    if isinstance(x, str):  
        previous_law = None  # 이전 법 이름 저장
        new_references = []

        for part in x.split(','):
            part = part.strip()
            part = re.sub(r'[\[\]［］].*?[\]\］]', '', part)
            part = re.sub(r'\s*[가-힣]\.\s*', '', part)
            part = part.strip('<br/>')
            part = part.replace('/', ',')
            part = re.sub(r'\s+', ' ', part).strip()

            # 현재 조각에 법 이름이 있는 경우, previous_law 업데이트
            for law_name in law_names:
                if law_name in part:
                    previous_law = law_name  # 이전 법 이름을 현재 법 이름으로 설정
                    break

            # 법 이름이 없고, 조를 포함하는 경우에만 이전 법 이름 붙이기
            if previous_law and not any(law_name in part for law_name in law_names):
                if re.search(r'\d+조', part):  # '조'를 포함하는 경우
                    part = f"{previous_law} {part}"

            new_references.append(part)

        # 결과 조각들을 다시 쉼표로 결합하여 반환
        return ', '.join(new_references)
    return x  # 문자열이 아닐 경우 그대로 반환

In [196]:
# 법과 시행령 이름을 붙이는 함수 / law_names_cleaned적용
def add_law_prefix1(x):
    if isinstance(x, str):  
        previous_law = None  # 이전 법 이름 저장
        new_references = []
        for part in x.split(','):
            part = part.strip()
            part = re.sub(r'[\[\]［］].*?[\]\］]', '', part)
            part = re.sub(r'\s*[가-힣]\.\s*', '', part)
            part = part.strip('<br/>')
            part = part.replace('/', ',')
            part = re.sub(r'\s+', ' ', part).strip()
        
            # 현재 조각에 법 이름이 있는 경우, previous_law 업데이트
            for law_name in law_names_cleaned:
                if law_name in part:
                    print(law_name)
                    previous_law = law_name  # 이전 법 이름을 현재 법 이름으로 설정
                    break

            # 법 이름이 없고, 조를 포함하는 경우에만 이전 법 이름 붙이기
            if previous_law and not any(law_name in part for law_name in law_names_cleaned):
                if re.search(r'\d+조', part):  # '조'를 포함하는 경우
                    part = f"{previous_law} {part}"

            new_references.append(part)

        # 결과 조각들을 다시 쉼표로 결합하여 반환
        return ', '.join(new_references)
    return x  # 문자열이 아닐 경우 그대로 반환

In [199]:
ccl[0]

'구 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것) 제91조 제1항 제3호, 제93조 제5호, 제11호 (차)목[현행 제93조 제10호 (타)목 참조], 법인세법 시행령 제132조 제2항 제7호, 행정소송법 제26조, [증명책임], 구 부가가치세법(2010. 1. 1. 법률 제9915호로 개정되기 전의 것) 제10조 제2항 제2호, (현행 제20조 제1항 제2호 참조), 제11조 제1항 제3호, (현행 제23조 제1항 참조), 제2항, (현행 제25조 제1항 참조), 구 부가가치세법 시행령(2013. 6. 28. 대통령령 제24638호로 전부 개정되기 전의 것) 제25조 제1항, (현행 제32조 제1항 참조), 행정소송법 제26조, [증명책임], '

In [198]:
ccl[0:2]

0    구 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것) 제91...
1        형법 제257조 제1항, 제258조의2 제1항, 제283조 제1항, 제284조, 
Name: 참조조문, dtype: object

In [197]:
reason1 = ccl[0:2].apply(lambda x: add_law_prefix1(x)) # 다처리
reason=ccl.apply(lambda x: add_law_prefix0(x)) #구 ~() 포함

법인세법
법인세법
행정소송법
부가가치세법
부가가치세법
행정소송법
형법


In [185]:
reason1

0       구 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것) 제91...
1       형법 제257조 제1항, 형법 제258조의2 제1항, 형법 제283조 제1항, 형법...
2       헌법 제12조, 형사소송법 제33조 제1항 제1호, 형사소송법 제69조, 형사소송법...
3       형사소송법 제294조의2 제1항, 제2항, 형사소송규칙 제134조의10 제1항, 형...
4       헌법 제13조 제1항, 형의 집행 및 수용자의 처우에 관한 법률 제107조, 형의 ...
                              ...                        
4184                                           형법 제161조, 
4185                                        형사소송법 제311조, 
4186                  형사소송법 제307조, 형사소송법 제308조, 형법 제10조, 
4187                                              참조조문 없음
4188    조치령 제1조, 제4조, 제4조 3호, 형사소송법 제516조, 형사소송법 제520조...
Name: 참조조문, Length: 4189, dtype: object

In [186]:
reason

0       구 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것) 제91...
1       형법 제257조 제1항, 형법 제258조의2 제1항, 형법 제283조 제1항, 형법...
2       헌법 제12조, 형사소송법 제33조 제1항 제1호, 형사소송법 제69조, 형사소송법...
3       형사소송법 제294조의2 제1항, 제2항, 형사소송규칙 제134조의10 제1항, 형...
4       헌법 제13조 제1항, 형의 집행 및 수용자의 처우에 관한 법률 제107조, 형의 ...
                              ...                        
4184                                           형법 제161조, 
4185                                        형사소송법 제311조, 
4186                  형사소송법 제307조, 형사소송법 제308조, 형법 제10조, 
4187                                              참조조문 없음
4188    조치령 제1조, 제4조, 제4조 3호, 형사소송법 제516조, 형사소송법 제520조...
Name: 참조조문, Length: 4189, dtype: object

In [97]:
import re

# 후처리 함수: 동법, 같은 법, 같은법을 이전 법 이름으로 대체
def replace_same_law_references(text):
    if isinstance(text, str):
        previous_law = None  # 직전 법 이름을 저장
        new_references = []

        for part in text.split(','):
            part = part.strip()
            # '동법', '같은 법', '같은법'이 포함된 경우 previous_law로 대체
            if any(keyword in part for keyword in ['동법', '같은 법', '같은법']):
                if previous_law:
                    part = re.sub(r'동법|같은\s*법|같은법', previous_law, part)
            else:
                # 새로운 법 이름이 있는 경우, previous_law 업데이트
                match = re.match(r'(.+?)(?:\s*제)?\s*\d+조', part)
                if match:
                    previous_law = match.group(1).strip()

            new_references.append(part)

        # 쉼표로 결합하여 반환
        return ', '.join(new_references)
    return text

In [101]:
final_result = reason.apply(replace_same_law_references)
final_result

0       구 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것) 제91...
1       형법 제257조 제1항, 형법 제258조의2 제1항, 형법 제283조 제1항, 형법...
2       헌법 제12조, 형사소송법 제33조 제1항 제1호, 형사소송법 제69조, 형사소송법...
3       형사소송법 제294조의2 제1항, 제2항, 형사소송규칙 제134조의10 제1항, 형...
4       헌법 제13조 제1항, 형의 집행 및 수용자의 처우에 관한 법률 제107조, 형의 ...
                              ...                        
4184                                           형법 제161조, 
4185                                        형사소송법 제311조, 
4186                  형사소송법 제307조, 형사소송법 제308조, 형법 제10조, 
4187                                              참조조문 없음
4188    조치령 제1조, 제4조, 제4조 3호, 형사소송법 제516조, 형사소송법 제520조...
Name: 참조조문, Length: 4189, dtype: object

In [102]:
final_result1 = reason1.apply(replace_same_law_references)
final_result1

0       구 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것) 제91...
1       형법 제257조 제1항, 형법 제258조의2 제1항, 형법 제283조 제1항, 형법...
2       헌법 제12조, 형사소송법 제33조 제1항 제1호, 형사소송법 제69조, 형사소송법...
3       형사소송법 제294조의2 제1항, 제2항, 형사소송규칙 제134조의10 제1항, 형...
4       헌법 제13조 제1항, 형의 집행 및 수용자의 처우에 관한 법률 제107조, 형의 ...
                              ...                        
4184                                           형법 제161조, 
4185                                        형사소송법 제311조, 
4186                  형사소송법 제307조, 형사소송법 제308조, 형법 제10조, 
4187                                              참조조문 없음
4188    조치령 제1조, 제4조, 제4조 3호, 형사소송법 제516조, 형사소송법 제520조...
Name: 참조조문, Length: 4189, dtype: object

In [132]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.max_colwidth')

# 참조판례 전처리

In [31]:
df['참조판례']=df['참조판례'].fillna('참조판례 없음')
#공백은 '참조판례 없음'으로 대체함.

In [32]:
#참조판례 정리

clean_part=[]
for part in df['참조판례']:
    part = part.strip()
    part = re.sub(r'[\[\]［］].*?[\]\］]', '', part)  # 대괄호와 그 안의 내용 제거
    part = re.sub(r'\s*[가-힣]\.\s*', '', part)  # '가.', '나.', '다.'와 같은 패턴 제거
    part = part.strip('<br/>')  # 공백 제거
    part = part.replace('/', ',')  # '/'를 ','로 변경
    part = re.sub(r'\s+', ' ', part).strip()
    clean_part.append(part)

df['참조판례']=clean_part




In [33]:
df['참조판례'].iloc[0]

'대법원 1994. 8. 12. 선고 92누12094 판결(공1994하, 2312), 대법원 2021. 2. 25. 선고 2017두237 판결, 대법원 2024. 4. 12. 선고 2023두58701 판결(공2024상, 800)'

In [34]:
#판례 참조 정도 확인

refered_jud=[]
for clean_part in df['참조판례']:
    parts = re.split(r',\s*(?![^()]*\))', clean_part)
    for sub in parts:
        refered_jud.append(sub)

refered_jud = [item for item in refered_jud if item.strip()]

refjud=pd.DataFrame(data=refered_jud)
refjud.value_counts()

0                                                                          
참조판례 없음                                                                        2281
대법원 2000. 7. 4. 선고 99도4341 판결(공2000하, 1851)                                       9
대법원 1994. 9. 13. 선고 94도1335 판결(공1994하, 2695)                                      8
대법원 2002. 6. 28. 선고 2002도2001 판결(공2002하, 1893)                                    8
대법원 1995. 5. 26. 선고 94다60509 판결(공1995하, 2258)                                     7
                                                                               ... 
대법원 1983.12.27. 선고 83도2557 판결                                                     1
대법원 1983.12.27. 선고 83다카644 판결(공1984,259)                                          1
대법원 1983.12.27 선고 83다카1503 판결                                                     1
대법원 1983.12.13. 선고 83다카975 판결(요특Ⅱ 자동차손해배상보장법 제3조(82)1222면 집31⑥민71 공721호163)       1
형사소송법 제310조(8)1452면)                                                              1


MySQL DB에 넣기.

1. PK를 판례일련번호로 하여서

2. 참조조문 정련 데이터, 참조조문 전처리 데이터, 참조판례 전처리 데이터를 열로 하는 테이블 작성

3. SQL에 올리기

In [35]:
dataF=pd.DataFrame(df['판례일련번호'])
dataF.head()
dataF['참조조문_1차전처리']=ccl
dataF['참조조문_전처리']=reason
dataF['참조판례_전처리']=df['참조판례']

In [37]:
dataF.to_csv('data/injury_case_clause_preprocessed.csv',index=False)

In [39]:
# db로 저장
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.dialects.mysql import LONGTEXT, TEXT
from db_info import user, password, host, port, database

####pandas dataframe에서는 불러올때 타입을 기본str로 불러와서 긴내용의 데이터삽입을 하려면 dtype을 설정해줘야함####
dtype = {
    '참조조문_1차전처리': TEXT,
    '참조조문_전처리': TEXT,
    '참조판례_전처리': TEXT
}
#판례내용_이유,판례내용_이유(전처리),판례내용_이유(불용어제거),클러스터,키워드

# DB정보
user = user
password = password
host = host
port = port
database = database

# CSV 파일 로드
df_injury_case_clause_preprocessed = pd.read_csv('data/injury_case_clause_preprocessed.csv')

# MySQL 데이터베이스 연결 설정
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

# 데이터프레임을 MySQL 테이블에 저장
df_injury_case_clause_preprocessed.to_sql('injury_case_clause_preprocessed', con=engine, if_exists='replace', index=False, dtype=dtype)
# 연결 닫기
engine.dispose()

print("데이터베이스 저장 완료")

데이터베이스 저장 완료
